In [14]:
import os
import openai
import sys
import langchain
from langchain_openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

langchain.debug=False

# Retrieval

### Loading Vector database

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embedding = OpenAIEmbeddings()

erasmus_vectordb = Chroma(persist_directory='./db/erasmus_website', embedding_function=embedding)

In [6]:
erasmus_vectordb.get()

{'ids': ['8ec48b98-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec4991c-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec47e50-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec47e64-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec470c2-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec470d6-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec470ea-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec470fe-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec47108-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec4711c-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec47130-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec47144-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec4714e-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec47162-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec47176-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec4718a-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec4719e-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec471a8-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec471bc-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec471d0-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec471e4-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec471ee-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec47202-dbda-11ee-ac22-f6fa68cb50d3',
  '8ec47216-dbda-11ee-ac22-

### Simple Retrieval

In [7]:
Question = 'Erasmus+ programı nedir / ne değildir?'
answers = erasmus_vectordb.similarity_search(Question, xf)

for ans in answers:
    print(ans.page_content)
    print('-------------------------------------------------------------------')

Source: https://erasmus.yildiz.edu.tr/sayfa/7/SSS-FAQ/807
Page Title:  Erasmus + Program Birimi Resmi Sitesi
Subject: SSS-FAQ

3- Erasmus+ programı bir “diploma” programı değildir.2.Erasmus, Erasmus+ farkı nedir?Erasmus 2014 yılı öncesinde Eğitim ve Gençlik Programları arasında yer alan bir alt başlıktı ve üniversite öğrencilerinin öğrenim hareketliliği programı anlamına geliyordu. 2014 yılında Avrupa Birliğinin Eğitim ve Gençlik alanındaki fırsatlarının çatı ismi Erasmus+ olarak değişti. 3.Başvuru koşulları nelerdir?Erasmus başvurusu için yükseköğretim kurumunda tam zamanlı öğrenci olmak, başvuruda kullanılmak üzere  AGNO (Ağırlıklı Genel Not Ortalaması) bulunması ve dil yeterliliğini belgelemek üzere dil sınav skoru gereklidir. Bu gereklilikler ile ilgili taban puanlar, her yıl Erasmus ofisi tarafından belirlenir ve başvuru çağrısı ile öğrencilere duyurulur. 4.Başvurular ne zaman?Başvurular gerçekleşecek faaliyetten bir akademik yıl önce alınır. Net başvuru tarihlerine, her akademik 

### Contextual Compression

<img src='https://miro.medium.com/v2/resize:fit:1400/1*RKPf3dTOR1VFsRHh81cNTw.png'/>

In [8]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor 

In [12]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

# Wrap our vectorstore
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=erasmus_vectordb.as_retriever()
)

In [15]:
Question = 'Erasmus+ programı nedir / ne değildir?'
answers = compression_retriever.get_relevant_documents(Question)
pretty_print_docs(answers)

/Users/mohammedbashir/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/mohammedbashir/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/mohammedbashir/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/mohammedbashir/anaconda3/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

- Erasmus+ programı bir “diploma” programı değildir.
- Erasmus, Erasmus+ farkı nedir?
- Erasmus 2014 yılı öncesinde Eğitim ve Gençlik Programları arasında yer alan bir alt başlıktı ve üniversite öğrencilerinin öğrenim hareketliliği programı anlamına geliyordu.
- 2014 yılında Avrupa Birliğinin Eğitim ve Gençlik alanındaki fırsatlarının çatı ismi Erasmus+ olarak değişti.
- Başvuru koşulları nelerdir?
- Erasmus başvurusu için yükseköğretim kurumunda tam zamanlı öğrenci olmak, başvuruda kullanılmak üzere  AGNO (Ağırlıklı Genel Not Ortalaması) bulunması ve dil yeterliliğini belgelemek üzere dil sınav skoru gereklidir.
- Bu gereklilikler ile ilgili taban puanlar, her yıl Erasmus
----------------------------------------------------------------------------------------------------
Document 2:

Erasmus+ Programı ile kişilere, yaş ve eğitim geçmişlerine bakılmaksızın yeni beceriler kazandırılması, onların kişisel gelişimlerinin güçlendirilmesi ve istihdam olanaklarının arttırılması a

### Self Query Retrieval

<img src='https://miro.medium.com/v2/resize:fit:1400/1*rrxKr3w0BoMZzB4nZioJyw.jpeg'/>

In [16]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo 

In [17]:
metadata_field_info = [
    AttributeInfo(
        name='Duyuru',
        description='An integer,1 or 0, that defines to define if the text is an anouncement / news/ haber/ duyuru or not.',
        type='integer',
    )
]

In [18]:
document_content_description = "YTU University Erasmus Website"
llm = OpenAI(temperature=0)
retriver = SelfQueryRetriever.from_llm(
    llm,
    erasmus_vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [20]:
Question = 'Erasmus+ programı nedir / ne değildir?'

answers = retriver.get_relevant_documents(Question)
for anw in answers:
    print(anw.page_content)
    print(f"Duyuru: {anw.metadata['Duyuru']}")
    print('----------------------------')


Source: https://erasmus.yildiz.edu.tr/haberler/626/2024-2025-Akademik-Yılı-Erasmus--KA-131-Programı-Öğrenci-Hareketlilikleri-Ön-Başvuru-Listeleri--05-01-2024-
Page Title:  Erasmus + Program Birimi Resmi Sitesi
Subject: 2024-2025-Akademik-Yılı-Erasmus--KA-131-Programı-Öğrenci-Hareketlilikleri-Ön-Başvuru-Listeleri--05-01-2024-

. Listeler hazırlanırken yüklenen Dil Belgeleri, Staj Davet Yazıları ve Dezavantaj Durum Belgeleri henüz kontrol edilmemiştir. Listelerimizdeki güncellemeler Süreç Takvimimizde belirtilen aşamalarla devam edecektir. Bu listede yer almış olmak, Erasmus+ Programına hak kazanıldığı anlamına gelmemektedir.Bazı Fakülte-Enstitü ve Bölüm adlarında öğrenci üniversite yerleştirme işlemleri öncesinde güncelleme yapılabilir.
Duyuru: 1
----------------------------
Source: https://erasmus.yildiz.edu.tr/haberler/633/2024-2025-Akademik-Yılı-Erasmus--Programı-Yabancı-Dil-Sınav-Listesi-Ön-İlanı-21-02-2024
Page Title:  Erasmus + Program Birimi Resmi Sitesi
Subject: 2024-2025-Akadem

### RA-Chain with Prompt

In [21]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [32]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Give a detailed answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=erasmus_vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)
result = qa_chain({'query': Question + 'Also append the website source of your info at the end'})

In [34]:
print(result['result'])
# print('---------')
# print(result['source_documents'])

Erasmus+ Programı, Avrupa Birliği tarafından eğitim ve gençlik alanında 2007-2013 yılları arasında uygulanmış olan Hayatboyu Öğrenme ve Gençlik Programlarının yerine uygulanacak olan yeni bir programdır. Bu program ağırlıklı olarak proje faaliyetlerine dayanmakla birlikte bireysel faaliyetleri de içermektedir. Ancak, Erasmus+ programı bir yabancı dil öğrenme programı veya bir burs programı değildir. (Source: https://erasmus.yildiz.edu.tr/sayfa/7/SSS-FAQ/807)


### Map Reduce Teknik

<img src='https://miro.medium.com/v2/resize:fit:1400/0*0zVwOYBM6Yg3Fdoe.jpg'/>

In [30]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=erasmus_vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain({'query': Question})

In [31]:
print(result['result'])
print('---------')
print(result['source_documents'])

Erasmus+ programı, Avrupa Birliği tarafından eğitim ve gençlik alanında uygulanan bir programdır. Bu program, öğrencilere yeni beceriler kazandırmayı, kişisel gelişimlerini güçlendirmeyi ve istihdam olanaklarını artırmayı amaçlar. Erasmus+ programı, bir diploma programı veya yabancı dil öğrenme programı değildir. Ayrıca, bir burs programı da değildir. Bu program genellikle proje faaliyetlerine dayanırken bireysel faaliyetleri de içerebilir.
---------
[Document(page_content='Source: https://erasmus.yildiz.edu.tr/sayfa/7/SSS-FAQ/807\nPage Title:  Erasmus + Program Birimi Resmi Sitesi\nSubject: SSS-FAQ\n\n3- Erasmus+ programı bir “diploma” programı değildir.2.Erasmus, Erasmus+ farkı nedir?Erasmus 2014 yılı öncesinde Eğitim ve Gençlik Programları arasında yer alan bir alt başlıktı ve üniversite öğrencilerinin öğrenim hareketliliği programı anlamına geliyordu. 2014 yılında Avrupa Birliğinin Eğitim ve Gençlik alanındaki fırsatlarının çatı ismi Erasmus+ olarak değişti. 3.Başvuru koşulları nel

## Query Enhacement

<img src='https://miro.medium.com/v2/resize:fit:928/1*QGjXF-1whUQYETVk6FA0ZQ.png'>

## RAG Triad

<img src='https://www.trulens.org/assets/images/RAG_Triad.jpg'>